In [1]:
from langchain_core.tools import tool


@tool
def multiplicar(a: int, b: int) -> int:
    """Multiplica dois números"""
    return a * b
 
multiplicar.args_schema.schema()

/tmp/ipykernel_38488/2026056485.py:9: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  multiplicar.args_schema.schema()


{'description': 'Multiplica dois números',
 'properties': {'a': {'title': 'A', 'type': 'integer'},
  'b': {'title': 'B', 'type': 'integer'}},
 'required': ['a', 'b'],
 'title': 'multiplicar',
 'type': 'object'}

In [2]:
from pydantic import BaseModel, Field
from typing import List


class CalculadoraLista(BaseModel):
    numeros: List[int] = Field(description="Lista de números")

@tool("multiplicar_lista", args_schema=CalculadoraLista, return_direct=True)
def multiplicar_lista(numeros: list) -> int:
    """Multiplica uma lista de números"""
    produto = 1
    for numero in numeros:
        produto = produto * numero
    return produto

multiplicar_lista.args_schema.schema()


/tmp/ipykernel_38488/20763493.py:16: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  multiplicar_lista.args_schema.schema()


{'properties': {'numeros': {'description': 'Lista de números',
   'items': {'type': 'integer'},
   'title': 'Numeros',
   'type': 'array'}},
 'required': ['numeros'],
 'title': 'CalculadoraLista',
 'type': 'object'}

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv

load_dotenv() 

llm = ChatOpenAI(model="gpt-4o-mini")
prompt = ChatPromptTemplate.from_template("Me conte uma piada sobre {topico}")
chain = prompt | llm | StrOutputParser()

chain_tool = chain.as_tool(name="Piadas", description="Conta uma piada sobre um determinado tópico")

/tmp/ipykernel_38488/3242570649.py:12: LangChainBetaWarning: This API is in beta and may change in the future.
  chain_tool = chain.as_tool(name="Piadas", description="Conta uma piada sobre um determinado tópico")


In [7]:
chain_tool.args_schema.schema()

{'properties': {'topico': {'title': 'Topico', 'type': 'string'}},
 'required': ['topico'],
 'title': 'PromptInput',
 'type': 'object'}

In [5]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=500)
tool = WikipediaQueryRun(api_wrapper=api_wrapper)

print(tool.invoke({"query": "langchain"}))

Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.




In [11]:
llm_wikipedia = llm.bind_tools([tool])
response = llm_wikipedia.invoke("o que é o langchain?")

In [12]:
response

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_xwzXpPkqfOwyu3fpB0lnthZ0', 'function': {'arguments': '{"query":"LangChain"}', 'name': 'wikipedia'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 88, 'total_tokens': 102, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_6fc10e10eb', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-4ff9a4a1-d04b-42c9-873b-024fb70ae3c9-0', tool_calls=[{'name': 'wikipedia', 'args': {'query': 'LangChain'}, 'id': 'call_xwzXpPkqfOwyu3fpB0lnthZ0', 'type': 'tool_call'}], usage_metadata={'input_tokens': 88, 'output_tokens': 14, 'total_tokens': 102, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoni

In [13]:
Wikipedia_chain = llm.bind_tools([tool]) | (lambda x: x.tool_calls[0]["args"]) | tool | StrOutputParser()

In [14]:
Wikipedia_chain.invoke("O que é o langchain")

"Page: LangChain\nSummary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.\n\n"

In [7]:
prompt_pos_tool = """
O usuário fez uma busca na Wikipedia que retornou o seguinte conteúdo:
{query}

Resuma para ele de um modo intuitivo
"""

from langchain_core.prompts import PromptTemplate
prompt_pos_tool = PromptTemplate.from_template(prompt_pos_tool)

In [16]:
Wikipedia_chain = llm.bind_tools([tool]) | (lambda x: x.tool_calls[0]["args"]) | tool | prompt_pos_tool | llm | StrOutputParser()

In [23]:
Wikipedia_chain.invoke("llamaindex")

'Um banco de dados vetorial é um tipo de banco de dados que armazena vetores, que são listas de números que representam dados em um espaço de muitas dimensões. Ele permite que você busque informações de forma eficiente, usando algoritmos que ajudam a encontrar rapidamente os dados mais próximos de um vetor de consulta. Em resumo, é uma ferramenta que facilita a busca e a organização de dados complexos de maneira rápida e eficaz.'

In [8]:
from langchain_core.tools import tool


@tool
def adicao(a: int, b: int) -> int:
    """Adiciona dois números"""
    return a + b


@tool
def multiplicacao(a: int, b: int) -> int:
    """Multiplica dois numeros"""
    return a * b


tools = [adicao, multiplicacao]

llm_tools= llm.bind_tools(tools)
llm_tools.invoke("Quanto é 12 + 7?").tool_calls[0]

{'name': 'adicao',
 'args': {'a': 12, 'b': 7},
 'id': 'call_khFKz99uZgM8WmnbAIIIHLoT',
 'type': 'tool_call'}

In [19]:
llm_sempre_multiplicacao = llm.bind_tools(tools,tool_choice="multiplicacao")
llm_sempre_multiplicacao.invoke("Quanto é 12 + 7?").tool_calls[0]

{'name': 'multiplicacao',
 'args': {'a': 12, 'b': 7},
 'id': 'call_nfgKjlyaObKqGzH8m9b6qZJC',
 'type': 'tool_call'}

In [20]:
llm_sempre_tool = llm.bind_tools(tools,tool_choice="any")
llm_sempre_tool.invoke("Olá, como você está hoje?").tool_calls[0]

{'name': 'adicao',
 'args': {'a': 5, 'b': 10},
 'id': 'call_U1XE8yeTMkeQxa3fLcrs5KcP',
 'type': 'tool_call'}

In [22]:
llm_sempre_tool = llm.bind_tools(tools)
llm_sempre_tool.invoke("Olá, como você está hoje?")

AIMessage(content='Olá! Estou aqui e pronto para ajudar. Como posso assisti-lo hoje?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 77, 'total_tokens': 94, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_6fc10e10eb', 'finish_reason': 'stop', 'logprobs': None}, id='run-33e6ba8c-6d12-41a1-83cb-d4cad1736d62-0', usage_metadata={'input_tokens': 77, 'output_tokens': 17, 'total_tokens': 94, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})